In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy.signal.windows as win

from speckit.dsp import timeshift
from speckit.core import SpectrumAnalyzer

# --- Noise Generation ---
N = int(1e6)  # samples
fs = 1e6  # Hertz
T_total = N / fs  # seconds
delay = 1e-4  # seconds
print(f"Total time series duration = {T_total} seconds")
print(f"Time delay = {delay} seconds")

# Generate base and time-shifted signals
np.random.seed(42)  # for reproducibility
y0 = np.random.normal(0, 1, N)
y1 = timeshift(y0, delay * fs)

# Truncate ends to remove boundary effects from the circular timeshift
n_trunc = int(2 * delay * fs)
if n_trunc > 0:
    y0 = y0[n_trunc:-n_trunc]
    y1 = y1[n_trunc:-n_trunc]

print(f"Total length of time series after cropping = {len(y1)}")

# --- Pre-configure the SpectrumAnalyzers ---
# We can create the analyzer objects once before the loop, which is more efficient.
# Analyzer for the reference case (zero delay)
analyzer_ref = SpectrumAnalyzer(data=[y0, y0], fs=fs, win=win.boxcar, olap=0.95)

# Analyzer for the delayed case
analyzer_delayed = SpectrumAnalyzer(data=[y0, y1], fs=fs, win=win.boxcar, olap=0.95)

# --- Main computation loop ---
coh_NBE = []
coh_dev = []
tf_NBE = []

# The x_axis represents the ratio of the fixed delay to the variable segment duration T
x_axis = np.linspace(0.01, 0.99, 40)  # Using 0.99 to avoid L=delay

# Using tqdm for the progress bar
for x in tqdm(x_axis, desc="Analyzing bias error"):
    # T is the variable segment duration in seconds
    T_segment = delay / x
    # L is the corresponding segment length in samples
    L = int(T_segment * fs)

    # We must ensure L does not exceed the total length of the cropped data
    if L >= len(y0):
        # This can happen if x is very small. Skip this iteration.
        coh_NBE.append(np.nan)
        coh_dev.append(np.nan)
        tf_NBE.append(np.nan)
        continue

    # Compute spectral estimates for a single bin at Nyquist frequency
    # Note: Using fs/2 is fine, but any high frequency works for this test.
    target_freq = fs / 2.0

    # Run computation for the reference case (zero delay)
    result_ref = analyzer_ref.compute_single_bin(freq=target_freq, L=L)

    # Run computation for the delayed case
    result_delayed = analyzer_delayed.compute_single_bin(freq=target_freq, L=L)

    # Extract scalar results using [0] and calculate Normalized Bias Error (NBE)
    coh_ref = result_ref.coh[0]
    coh_delayed = result_delayed.coh[0]

    # Use .cf for the magnitude of the transfer function Hxy
    tf_mag_ref = result_ref.cf[0]
    tf_mag_delayed = result_delayed.cf[0]

    coh_NBE.append((coh_delayed - coh_ref) / coh_ref)
    coh_dev.append(result_delayed.coh_dev[0])
    tf_NBE.append((tf_mag_delayed - tf_mag_ref) / tf_mag_ref)

# --- Plotting the results ---
fig, ax = plt.subplots(figsize=(5, 4), dpi=150)

# Theoretical NBE curves from Bendat & Piersol
# NBE of frequency response estimate (https://doi.org/10.1016/0022-460X(78)90396-6, Equation 23)
ax.plot(
    x_axis,
    (1 - x_axis) - 1,
    c="gray",
    ls="-",
    lw=6,
    alpha=0.5,
    label="Theoretical TF Mag NBE",
)
# NBE of coherence estimate (https://doi.org/10.1016/0022-460X(78)90396-6, Equation 24)
ax.plot(
    x_axis,
    (1 - x_axis) ** 2 - 1,
    c="gray",
    ls="--",
    lw=6,
    alpha=0.5,
    label="Theoretical Coherence NBE",
)

# Plot simulated results
ax.plot(
    x_axis,
    tf_NBE,
    "o",
    color="k",
    markerfacecolor="none",
    label=r"Simulated $|H_{xy}|$ NBE",
)
ax.plot(x_axis, coh_NBE, "x", color="crimson", label=r"Simulated $\gamma_{xy}^2$ NBE")

ax.set_xlabel(r"Delay / Segment Duration ($\tau_0 / T$)")
ax.set_ylabel(r"Normalized Bias Error")
ax.set_title("Time Delay Bias Error in Spectral Estimates")
ax.set_xlim(0, 1)
ax.set_ylim(-1, 0.05)
ax.grid(True, which="both", linestyle=":")
ax.legend(loc="best", fontsize=8)
fig.tight_layout()
plt.show()